## Argonne Covid Database

This notebook will get you started with making requests to the Argonne Covid Database. You will need a few items to get started:

1. A whitelisted IP
    * Currently, only users from LLNL, Argonne, and BNL may make requests to this database.
1. A user account
    * You must ask for a user account before you can make queries.
    
The first few notebooks will ensure you can access the database. After that, you will be guided through a range of simple queries.

In [ ]:
import requests
import json
import urllib3
import getpass
import hashlib

COVID_URL = 'https://covid-ws-01.alcf.anl.gov'
RQARGS = {
    # Connections are only available from select IP Addresses, timeout after a few seconds.
    'timeout': 3,
    # Certs are not yet setup for the above host
    'verify': False
}

In [ ]:
try:
    requests.get(COVID_URL, **RQARGS)
    print('Connection Successful, please proceed')
except requests.exceptions.Timeout:
    print(f'Connection Failed, {COVID_URL} is only accessible '
          'from select ip ranges at LLNL, Argonne, and BNL. Please try again using a different IP.')

In [ ]:
payload = {
    'email': input('Enter your email (EX: person@anl.gov) > '), 
    'password': getpass.getpass(),
}
token = ''
response = requests.post(f'{COVID_URL}/rpc/login', data=payload, **RQARGS)
if response != 200:
    print(f'Login Failed with {payload["email"]}: {response.json()["message"]}')
    print('Please send the message above to your administrator.')
else:
    print('Authentication Successful, you may now make requests to the database.')
    token = response.json()[0]['token']
headers = {'Authorization': f'bearer {token}'}

In [ ]:
smile = """CCC(COC(=O)C(NP(=O)(Oc1ccccc1)OCC1OC(C(C1O)O)(C#N)c1ccc2n1ncnc2N)C)CCCCN(CCCC(Nc1ccnc2c1ccc(c2)Cl)C)CCCC(=O)Oc1ccccc1C(=O)Nc1ncc(s1)[N+](=O)[O-]NC(=Nc1ccc(cc1)C(=O)Oc1ccc2c(c1)ccc(c2)C(=N)N)N"""
params = {
    'md5': 'eq.{}'.format(hashlib.md5(smile.encode()).hexdigest())
}
requests.post(f'{COVID_URL}/dup_m2k', params=params, headers=headers, **RQARGS).json()